In [1]:
import ee
import collections
collections.Callable = collections.abc.Callable
import geemap
from geemap import chart

%matplotlib inline
import matplotlib.pyplot as plt

ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AX4XfWj1ID_E7IT3UMWCx06hcJgP903AcYPUlLvJJVlrTYlf1U8TMEO4Q5s



Successfully saved authorization token.


## Imports & Constants

In [2]:
import ee
import geemap

geometry = ee.Geometry.Polygon(
        [[[25.67758947139908, -10.52500539578282],
          [25.67758947139908, -10.831343855184292],
          [26.285562854675604, -10.831343855184292],
          [26.285562854675604, -10.52500539578282]]], None, False)
mines = ee.FeatureCollection("users/EmilyNason/cod_mines_curated_all_opendata_p_ipis")
s2 = ee.ImageCollection("COPERNICUS/S2_SR")
s1 = ee.ImageCollection('COPERNICUS/S1_GRD')

area1 = ee.Geometry.Polygon(
        [[[25.783982219615126, -10.760592214819402],
          [25.783982219615126, -10.804773403670321],
          [25.882344188609267, -10.804773403670321],
          [25.882344188609267, -10.760592214819402]]], None, False)
area2 = ee.Geometry.Polygon(
        [[[25.883463192760367, -10.653415341387422],
          [25.883463192760367, -10.684961077984651],
          [25.918653775035757, -10.684961077984651],
          [25.918653775035757, -10.653415341387422]]], None, False)
area3 = ee.Geometry.Polygon(
        [[[26.117306035200816, -10.551319560770521],
          [26.117306035200816, -10.632650522900946],
          [26.248455327193003, -10.632650522900946],
          [26.248455327193003, -10.551319560770521]]], None, False)
area4 = ee.Geometry.Polygon(
        [[[25.961694541782418, -10.617121740523016],
          [25.961694541782418, -10.656262872369753],
          [26.00804311355976, -10.656262872369753],
          [26.00804311355976, -10.617121740523016]]], None, False)
noMine = ee.Geometry.Polygon(
        [[[26.05587044825878, -10.73615043816951],
          [26.05587044825878, -10.787712996362622],
          [26.14122124360419, -10.787712996362622],
          [26.14122124360419, -10.73615043816951]]], None, False)
focus = ee.Geometry.Polygon(
[[[27.350233348102517, -7.518171474050515],
[27.350233348102517, -7.57841301205225],
[27.436407359332986, -7.57841301205225],
[27.436407359332986, -7.518171474050515]]], None, False)

rgbVis = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 3000,
}

waterVis = {
    'min':0, 
    'max':1, 
    'palette': ['blue']
}

miningVis = {
    'min':0, 
    'max':1, 
    'palette': ['red']
}

legend_dict = {
    'Water': ('0000FF'), # blue
    'Mine Estimate': ('FF0000'), # red
}

## True Color Composites (Sentinel-2)

In [3]:
def create_median_composites(geometry, start, end):
    images = []
    
    for year in range(start, end+1):
        filtered = s2 \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
            .filter(ee.Filter.intersects('.geo', geometry)) \
            .filter(ee.Filter.date(str(year) + '-01-01', str(year) + '-12-31')) \

        composite = filtered.median().clip(geometry)
        
        images.append((composite, year))
        
    return images

def add_median_composites(images, geometry):
    Map = geemap.Map()
    Map.centerObject(geometry, 11)
    
    for image, year in images:
        Map.addLayer(image, rgbVis, str(year) + ' Median Composite')
        
    return Map

In [4]:
images = create_median_composites(geometry, 2019, 2021)
Map = add_median_composites(images, geometry)
Map

Map(center=[-10.67819783565169, 25.98157616303779], controls=(WidgetControl(options=['position', 'transparent_…

## SAR Composites

In [5]:
def create_sar_composites(geometry, start, end):
    images = []
    
    for year in range(start, end+1):
        filtered = s1 \
            .filter(ee.Filter.eq('instrumentMode','IW')) \
            .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
            .filter(ee.Filter.eq('resolution_meters',10)) \
            .filter(ee.Filter.intersects('.geo', geometry)) \
            .filter(ee.Filter.date(str(year) + '-01-01', str(year) + '-12-31')) \
            .select(['VV', 'VH'])

        composite = filtered.median().clip(geometry)
        
        vv_vh = composite.select('VV').divide(composite.select('VH')).rename('VV/VH')
        composite.addBands(vv_vh, ['VV/VH']).rename('VV/VH')
        
        images.append((composite, year))
        
    return images

In [6]:
imagesS1 = create_sar_composites(geometry, 2019, 2021)

## VH Band

In [7]:
def add_vh_composites(images, imagesVH, geometry):
    Map = geemap.Map()
    Map.centerObject(geometry, 11)
    
    for image, year in images:
        Map.addLayer(image, rgbVis, str(year) + ' Median Composite')
    
    for image, year in imagesVH:
        Map.addLayer(image.select('VH'), {'min':-25, 'max':0}, str(year) + ' VH Composite')
        
    return Map

In [8]:
Map = add_vh_composites(images, imagesS1, geometry)
Map

Map(center=[-10.67819783565169, 25.98157616303779], controls=(WidgetControl(options=['position', 'transparent_…

### VH Calculations

In [9]:
def create_vh_calcs(images, geometry):
    Map = geemap.Map()
    Map.centerObject(geometry, 11)
    
    for image, year in images:
        
        filtered = s2 \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
            .filter(ee.Filter.intersects('.geo', geometry)) \
            .filter(ee.Filter.date(str(year) + '-01-01', str(year) + '-12-31')) \

        composite = filtered.median().clip(geometry)
        
        Map.addLayer(composite, rgbVis, str(year) + ' Median Composite')
        
        ndvi = composite.normalizedDifference(['B8', 'B4'])
        nirG = composite.normalizedDifference(['B8', 'B3'])
        ndmi = composite.normalizedDifference(['B8', 'B11'])
        
        image = image.select('VH')
        
        waterStorage = image.lt(-22).rename('water').And(nirG.lt(0.1).Or(ndmi.gt(.3))).selfMask()
        connectW = waterStorage.connectedPixelCount(25)
        waterStorage = waterStorage.updateMask(connectW.gt(8))
        Map.addLayer(waterStorage, waterVis, 'Water ' + str(year))
        
        mining = image.gte(-22).And(image.lt(-18)).And(ndvi.lt(.15)).rename('mine').selfMask()
        connectM = mining.connectedPixelCount(25)
        mining = mining.updateMask(connectM.gt(8))
        Map.addLayer(mining, miningVis, 'Mining Estimate ' + str(year))
        
    return Map

In [10]:
Map = create_vh_calcs(imagesS1, geometry)
Map.add_legend(legend_title='Legend', legend_dict=legend_dict, position='bottomright')
Map

Map(center=[-10.67819783565169, 25.98157616303779], controls=(WidgetControl(options=['position', 'transparent_…

## VV Band

In [11]:
def add_vv_composites(images, imagesVV, geometry):
    Map = geemap.Map()
    Map.centerObject(geometry, 11)
    
    for image, year in images:
        Map.addLayer(image, rgbVis, str(year) + ' Median Composite')
    
    for image, year in imagesVV:
        Map.addLayer(image.select('VV'), {'min':-25, 'max':0}, str(year) + ' VV Composite')
        
    return Map

In [12]:
Map = add_vv_composites(images, imagesS1, geometry)
Map

Map(center=[-10.67819783565169, 25.98157616303779], controls=(WidgetControl(options=['position', 'transparent_…

### VV Calculations

In [31]:
def create_vv_calcs(images, geometry):
    Map = geemap.Map()
    Map.centerObject(geometry, 11)
    
    for image, year in images:
        filtered = s2 \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
            .filter(ee.Filter.intersects('.geo', geometry)) \
            .filter(ee.Filter.date(str(year) + '-01-01', str(year) + '-12-31')) \

        composite = filtered.median().clip(geometry)
        
        Map.addLayer(composite, rgbVis, str(year) + ' Median Composite')
        
        ndvi = composite.normalizedDifference(['B8', 'B4'])
        nirG = composite.normalizedDifference(['B8', 'B3'])
        ndmi = composite.normalizedDifference(['B8', 'B11'])
        
        image = image.select('VV')
        
        waterStorage = image.lt(-15).rename('water').And(nirG.lt(0.1).Or(ndmi.gt(.3))).selfMask()
        connectW = waterStorage.connectedPixelCount(25)
        waterStorage = waterStorage.updateMask(connectW.gt(8))
        Map.addLayer(waterStorage, waterVis, 'Water ' + str(year))
        
        mining = image.gte(-15).And(image.lt(-9)).And(ndvi.lt(.15)).rename('mine').selfMask()
        connectM = mining.connectedPixelCount(25)
        mining = mining.updateMask(connectM.gt(8))
        Map.addLayer(mining, miningVis, 'Mining Estimate ' + str(year))
        
    return Map

In [32]:
Map = create_vv_calcs(imagesS1, geometry)
Map.add_legend(legend_title='Legend', legend_dict=legend_dict, position='bottomright')
Map

Map(center=[-10.67819783565169, 25.98157616303779], controls=(WidgetControl(options=['position', 'transparent_…

## VV/VH Band

In [15]:
def add_vv_vh_composites(images, imagesRatio, geometry):
    Map = geemap.Map()
    Map.centerObject(geometry, 11)
    
    for image, year in images:
        Map.addLayer(image, rgbVis, str(year) + ' Median Composite')
    
    for image, year in imagesRatio:
        Map.addLayer(image.select('VV').divide(image.select('VH')), {}, str(year) + ' VV/VH')
        
    return Map

In [16]:
Map = add_vv_vh_composites(images, imagesS1, geometry)
Map

Map(center=[-10.67819783565169, 25.98157616303779], controls=(WidgetControl(options=['position', 'transparent_…

### VV/VH Calculations

In [33]:
def create_vv_vh_calcs(images, geometry):
    Map = geemap.Map()
    Map.centerObject(geometry, 11)
    
    for image, year in images:
        
        filtered = s2 \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) \
            .filter(ee.Filter.intersects('.geo', geometry)) \
            .filter(ee.Filter.date(str(year) + '-01-01', str(year) + '-12-31')) \

        composite = filtered.median().clip(geometry)
        
        Map.addLayer(composite, rgbVis, str(year) + ' Median Composite')
        
        ndvi = composite.normalizedDifference(['B8', 'B4'])
        nirG = composite.normalizedDifference(['B8', 'B3'])
        ndmi = composite.normalizedDifference(['B8', 'B11'])

        waterStorage = image.select('VV').divide(image.select('VH')).gt(0.6).And(nirG.lt(0.1).Or(ndmi.gt(.3))).rename('water').selfMask()
        connectW = waterStorage.connectedPixelCount(25)
        waterStorage = waterStorage.updateMask(connectW.gt(8))
        Map.addLayer(waterStorage, waterVis, 'Water ' + str(year))
        
        #currently not doing much -> basically going off of NDVI
        mining = image.select('VV').divide(image.select('VH')).lt(0.6).And(ndvi.lt(.15)).rename('mine').selfMask()
        connectM = mining.connectedPixelCount(25)
        mining = mining.updateMask(connectM.gt(8))
        Map.addLayer(mining, miningVis, 'Mining Estimate ' + str(year))
        
    return Map

In [34]:
Map = create_vv_vh_calcs(imagesS1, geometry)
Map.add_legend(legend_title='Legend', legend_dict=legend_dict, position='bottomright')
Map

Map(center=[-10.67819783565169, 25.98157616303779], controls=(WidgetControl(options=['position', 'transparent_…

## VH/VV Band

In [39]:
def add_vh_vv_composites(images, imagesRatio, geometry):
    Map = geemap.Map()
    Map.centerObject(geometry, 11)
    
    for image, year in images:
        Map.addLayer(image, rgbVis, str(year) + ' Median Composite')
    
    for image, year in imagesRatio:
        Map.addLayer(image.select('VH').divide(image.select('VV')), {}, str(year) + ' VV/VH')
        
    return Map

In [40]:
Map = add_vh_vv_composites(images, imagesS1, geometry)
Map

Map(center=[-10.67819783565169, 25.98157616303779], controls=(WidgetControl(options=['position', 'transparent_…

## VH, VV, VH/VV False Color Composite

In [43]:
def add_false_composites(images, imagesRatio, geometry):
    Map = geemap.Map()
    Map.centerObject(geometry, 11)
    
    for image, year in images:
        Map.addLayer(image, rgbVis, str(year) + ' Median Composite')
    
    for image, year in imagesRatio:
        Map.addLayer(image, {'bands': ['VV', 'VH'],'min': -25,'max': -5}, str(year) + ' False Color Composite')
        # red VH, blue VV
    return Map

In [44]:
Map = add_false_composites(images, imagesS1, geometry)
Map

Map(center=[-10.67819783565169, 25.98157616303779], controls=(WidgetControl(options=['position', 'transparent_…